In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

int(config['augmentations']['sample_rate'])

In [1]:
from model import ResNet18
import torch, torchaudio
from AudioDataset import AudioDataset
from pathlib import Path
# load back the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn = ResNet18.to(device)

model_paths = [str(p) for p in Path('./saved_model/').glob(f'*.pt')]
for i, model_path in enumerate(model_paths):
    print(f'[{i}] {model_path}')

path = model_paths[int(input('Select saved model > '))]
state_dict = torch.load(path, map_location=device)['model_state_dict']
cnn.load_state_dict(state_dict)

torch.save(cnn, './saved_model/NoAugmentations44K.pt')


C:\Users\Jian Quan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in C:\Users\Jian Quan/.cache\torch\hub\pytorch_vision_v0.10.0


[0] saved_model\Augmentations8K.pt
[1] saved_model\NoAugmentations44K.pt
[2] saved_model\NoAugmentations44K_cp1.pt
[3] saved_model\NoAugmentations8K.pt


In [ ]:
import matplotlib.pyplot as plt
dataset = AudioDataset(['test/ac.wav'])
testData = torch.unsqueeze(dataset[0][0], 0)
plt.imshow(testData[0][0])

In [ ]:
import matplotlib.pyplot as plt
import Augmentation
from AudioDataset import AudioDataset
import torch
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')


audio_paths = Augmentation.getAudio('./data/testset')
audio_test_dataset = AudioDataset(audio_paths)

train_dataloader = torch.utils.data.DataLoader(
    audio_test_dataset,
    batch_size=int(config['model']['batch_size']),
    num_workers=int(config['model']['num_workers']),
    shuffle=True,
    pin_memory=True,
)
